In [1]:
import os, sys, time
from time import sleep
from pathlib import Path
import fnmatch, glob, shutil
import sched, tempfile, textwrap
from zipfile import ZipFile

In [2]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame

In [3]:
import email, smtplib, ssl
import os.path as op
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.utils import COMMASPACE, formatdate

In [4]:
import threading, logging
from threading import Timer
from PyPDF2 import PdfFileReader, PdfFileWriter
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

## *`global` vars*

In [5]:
# BOOL 
global eod
eod = False

In [6]:
global out_directory

## *DEFINE CLASSES*

In [7]:
class CofA_Thread(threading.Thread): #{
    
    def __init__(self, threadID, name, counter): #{
        threading.Thread = threadID
        self.threadID = threadID
        self.name = name
        self.counter = counter
    #}
    
    def run(self): #{
        logging.info("Starting " + self.name)
        print_time(self.name, self.counter, 5)
        logging.info("Exiting " + self.name)
    #}
#}

In [8]:
class CofA_Event_Handler(FileSystemEventHandler): #{
    
    def __init__(self, save_list, temp_dir, out_dir): #{
        self.save_list = save_list
        #self.in_dir = in_dir
        self.temp_dir = temp_dir
        self.out_dir = out_dir
    #}
    
    def dispatch(self, event): #{
        # INSTANTIATE GLOBAL VARS
        global out_directory
        ts = pd.Timestamp.now()
        #print("e=" + str(event))
        #logging.info("time=" + str(ts))
        output_str = "EVENT >> " + str(event.event_type)
        output_str += str("\t\t\t\t\t [is_dir] == " + str(event.is_directory))  # WAS:  + " |"
        output_str += str("\t\t\t\t\t " + str(event.src_path))
        logging.info(textwrap.fill(output_str,
                                  initial_indent='',
                                  subsequent_indent='   ' * 4,
                                  width=55
                                  ))
        #logging.info("NEW FILE NAME == " + str(new_name)) # NOT NEEDED, is within above function
        """
        # new location var?
        #new_loc = os.path.join(out_dir, new)
        """
        """
        copy_and_move(the_src_path=event.src_path,
                     the_dst_path=temp_dir,
                     new_file_name=new_name)
        """
        
        if (str(event.event_type) == "created") | (str(event.event_type) == "modified"): #{
            print("CREATE/MODIFY")
            the_event_path = Path(event.src_path)
            
            
            # CREATE file_name VAR
            file_name = os.path.basename(the_event_path)
            # CREATE TUPLE
            event_tuple = (str(file_name), str(ts))
            # APPEND (tuple) TO SAVE_LIST
            save_list.append(event_tuple)
            # WATERMARK/MOVE PDF VIA FUNCTION
            move_newly_created_pdfs(the_src_path=the_event_path,
                                    the_dst_path=out_directory)
        #}        
        elif (str(event.event_type) == "deleted"): #{
            print("DELETE")
        #}
        
    #}
    
    def on_created(self, event): #{
        ts = pd.Timestamp.now()
    #}
    
    def on_modified(self, event): #{
        logging.info("<<MODIFIED FUNCTION>>")
    #}
    
    def on_deleted(self, event): #{
        logging.info("<<DELTED FUNCTION>>")
        # GET FILE NAME FROM DELETED FILE
        del_name = os.path.basename(event.src_path)
        # CREATE PATH VAR AND LIKEWISE DELETE
        del_path = os.path.join(out_dir, del_name)
        # CHECK IF ALREADY DELETED:
        if os.path.exists(del_path): #{
            # EXISTS SO DELETE!
            os.remove(del_path)
            logging.info(str(del_path) + " DELETED !")
        #}
        else: #{
            # ALREADY DELETED
            logging.info("already deleted!")
        #}
    #}   
#}

## *DEFINE FUNCTIONS*

In [9]:
def test_hello(): #{
    logging.info('hello, world!')
    logging.info(threading.enumerate())
#}

In [10]:
def create_watermark(input_pdf, output, watermark): # {
    try:  # {
        watermark_obj = PdfFileReader(watermark)
        watermark_page = watermark_obj.getPage(0)

        pdf_reader = PdfFileReader(input_pdf)
        pdf_writer = PdfFileWriter()

        # Watermark all the pages
        for page in range(pdf_reader.getNumPages()):  # {
            page = pdf_reader.getPage(page)
            page.mergePage(watermark_page)
            pdf_writer.addPage(page)
        # }

        with open(output, 'wb') as out:  # {
            pdf_writer.write(out)
        # }
    # }
    except: # {
        errorMessage = str(sys.exc_info()[0]) + "\n\t\t"
        errorMessage = errorMessage + str(sys.exc_info()[1]) + "\n\t\t"
        errorMessage = errorMessage + str(sys.exc_info()[2]) + "\n"
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        typeE = str("TYPE : " + str(exc_type))
        fileE = str("FILE : " + str(fname))
        lineE = str("LINE : " + str(exc_tb.tb_lineno))
        messageE = str("MESG : " + "\n" + str(errorMessage) + "\n")
        logging.error("\n" + typeE +
              "\n" + fileE +
              "\n" + lineE +
              "\n" + messageE)
        # }
    else: # {
        logging.info("[watermark-pdf] FIN...")
    # }
    return
# }

In [11]:
"""
MOVE NEWLY CREATED .PDFs to 'TEMP' in order to be WATERMARKED
"""
def move_newly_created_pdfs(the_src_path, the_dst_path): #{
    # GET NEWLY_MADE 'file_name'
    file_name = os.path.basename(the_src_path)
    logging.info("FILE_NAME >>> " + str(file_name))
    # CHECK IF FILE IS OF .PDF
    if fnmatch.fnmatch(file_name, "*.pdf"): #{
        # CREATE TEMP DIR TO WORK INSIDE OF:
        with tempfile.TemporaryDirectory() as directory_name: #{
            the_dir = Path(directory_name)
            logging.info("TEMP_DIRECTORY >>> " + str(the_dir))
            # COPY NEWLY_MADE FILE INTO TEMP_DIR
            shutil.copy(src=Path(the_src_path), dst=the_dir)
            # create temp_file_path from within TEMP_DIR
            temp_path = os.path.join(the_dir, file_name)
            # PERFORM STRING OPERATIONS
            #################################
            idx_mrk = file_name.rfind('@', 0, len(file_name))
            half1 = str(file_name[0:idx_mrk])
            half2 = str(file_name[idx_mrk + 1:len(file_name)])
            logging.info("HALF 1 == " + half1)
            logging.info("HALF 2 == " + half2)
            #  setup NEW FILE NAME (for copy)
            new_name = "part "
            new_name += str(half1)
            new_name += " CofA Lot# "
            new_name += str(half2)
            logging.info("NEW NAME == " + str(new_name))
            #################################
            # CREATE 'new_path' by using "new_name"
            new_path = os.path.join(out_directory, new_name)
            # CREATE WATERMARK AND MOVE FILES OUT OF TEMP_DIR
            create_watermark(input_pdf=temp_path,
                            output=new_path,
                            watermark=in_file)
        #}
        logging.info("Name of temp_dir: " + str(the_dir))
        logging.info("Directory exists after? " + str(the_dir.exists()))
        logging.info("Contents after:" + str(list(the_dir.glob('*'))))
    #}
    else: #{
        logging.info("NOT A .PDF!")
    #}
#}

In [12]:
def send_mail(send_from, send_to, subject, message, files=[],
             server="cos.smtp.agilent.com", port=587, use_tls=True): #{
    print("SEND MAIL")
#}

---
# *`main()` function call*

In [13]:
if __name__ == "__main__": #{
    logging.basicConfig(level=logging.INFO,
                       stream=sys.stdout,
                       format='%(asctime)s: \n%(message)s\t<%(threadName)s-ID:%(thread)d(%(funcName)s)LINE:%(lineno)s>',
                       datefmt='%Y-%m-%d-%H%M%S')
    in_file = "C:/CofA/imp/Agilent_CofA_Letterhead_03-21-19.pdf"
    """
    in_directory = "F:/APPS/CofA/"
    out_directory = "G:/C of A's/#Email Node/"
    """
    in_directory = "C:/Temp/F/APPS/CofA/"
    out_directory = "C:/Temp/G/C of A's/#Email Node/"
    save_list = []
    logging.info("PRINT BEFORE TIMER STARTS !!")
    # SETUP TIMER
    t = Timer(15.0, test_hello) # 2592000
    # START TIMER
    t.start()
    with tempfile.TemporaryDirectory() as temporary_directory: #{
        logging.info("created temp_dir")
        temp_dir = str(temporary_directory)
        logging.info("dir name: " + str(temp_dir))
        # create event handler
        event_handler = CofA_Event_Handler(save_list=save_list,
                                          temp_dir=temp_dir,
                                          out_dir=out_directory)
        observer = Observer()
        observer.schedule(event_handler=event_handler,
                         path=in_directory,
                         recursive=True)
        observer.start()
        try: #{
            while True: #{
                sleep(1)
            #}
        #}
        except KeyboardInterrupt: #{
            observer.stop()
        #}
        observer.join()
    #}
    logging.info("exists after: " + str(os.path.exists(temp_dir)))
#}

2019-08-05-150709: 
PRINT BEFORE TIMER STARTS !!	<MainThread-ID:98492(<module>)LINE:14>
2019-08-05-150709: 
created temp_dir	<MainThread-ID:98492(<module>)LINE:20>
2019-08-05-150709: 
dir name: C:\Users\derbates\AppData\Local\Temp\tmp22kgvw68	<MainThread-ID:98492(<module>)LINE:22>
2019-08-05-150714: 
EVENT >> created
            [is_dir] == False
            C:/Temp/F/APPS/CofA/ICP-079@0006479330.pdf	<Thread-7-ID:102492(dispatch)LINE:22>
CREATE/MODIFY
2019-08-05-150714: 
FILE_NAME >>> ICP-079@0006479330.pdf	<Thread-7-ID:102492(move_newly_created_pdfs)LINE:7>
2019-08-05-150714: 
TEMP_DIRECTORY >>> C:\Users\derbates\AppData\Local\Temp\tmpjvg5adhd	<Thread-7-ID:102492(move_newly_created_pdfs)LINE:13>
2019-08-05-150714: 
HALF 1 == ICP-079	<Thread-7-ID:102492(move_newly_created_pdfs)LINE:23>
2019-08-05-150714: 
HALF 2 == 0006479330.pdf	<Thread-7-ID:102492(move_newly_created_pdfs)LINE:24>
2019-08-05-150714: 
NEW NAME == part ICP-079 CofA Lot# 0006479330.pdf	<Thread-7-ID:102492(move_newly_crea

In [14]:
save_list

[('ICP-079@0006479330.pdf', '2019-08-05 15:07:14.019025'),
 ('AMN-603-1@0006480057.pdf', '2019-08-05 15:07:25.518228')]

In [15]:
save_df = pd.DataFrame(data=save_list, columns=['CofA_File_Name', 'Time_Stamp'])

In [16]:
save_df

,CofA_File_Name,Time_Stamp
0,ICP-079@0006479330.pdf,2019-08-05 15:07:14.019025
1,AMN-603-1@0006480057.pdf,2019-08-05 15:07:25.518228
